### Connect to the database

In [ ]:
import sqlite3
import geopandas as gpd
import json
from datetime import datetime
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap

conn = sqlite3.connect('../max-experiments/itinerary-scraping/journeys.db')
cursor = conn.cursor()

# display all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

### Import Scraped Data

In [ ]:
cursor.execute('SELECT * FROM outages')
outages = cursor.fetchall()

cursor.execute('SELECT * FROM journeys')
journeys = cursor.fetchall()

cursor.execute('SELECT * FROM stops')
stops = cursor.fetchall()

cursor.close()
conn.close()

### Set Up Dataframes

In [ ]:
outages_df = pd.DataFrame(outages, columns=['outage_id', 'stop_id', 'effect', 'updated_at', 'outage_data'])
stops_df = pd.DataFrame(stops, columns=['stop_id', 'stop_name', 'stop_coords', 'stop_acc'])

# merge outages_df with stops_df
merged_df = pd.merge(outages_df, stops_df, on='stop_id')

### Import TCL Data

In [ ]:
tcl_metro = gpd.read_file('data/tcl_metro.json')
tcl_metro = tcl_metro.drop(columns=['date_debut', 'date_fin', 'last_update', 'last_update_fme'])

In [ ]:
# Metro line colors
colors = []
for index, row in tcl_metro.iterrows():
    if row['ligne'] == 'D': # green line
        colors.append('#029C41')
    elif row['ligne'] == 'A':   # pink line
        colors.append('#E62E86')
    elif row['ligne'] == 'B':   # blue line
        colors.append('#0065B1')
    elif row['ligne'] == 'C':   # yellow line
        colors.append('#F48E06')
    else:   # furnicular line, light green
        colors.append("#93BF38")
        
tcl_metro['style'] = [
    {
        "color": colors[i],
        "weight": 5,
        "opacity": 1
    }
    for i in range(len(tcl_metro))
]

### Expand Condensed Data

In [ ]:
# unpack stop_coords
merged_df['lon'] = merged_df['stop_coords'].apply(lambda x: x.split(',')[0][9:-1])
merged_df['lat'] = merged_df['stop_coords'].apply(lambda x: x.split(',')[1][9:-2])
merged_df = merged_df.drop(columns=['stop_coords'])

# unpack outage_data
merged_df['outage_data'] = merged_df['outage_data'].apply(lambda x: json.loads(x))
merged_df['stop_acc'] = merged_df['stop_acc'].apply(lambda x: json.loads(x))
begin, end = [], []
cause, effect = [], []
for row in merged_df['outage_data']:
    ### Start and End Times
    # current format: YYYYMMDDTHHMMSS
    # required format: YYYY-MM-DDTHH:MM:SS+02:00        
    times = row['periods']    
    begin.append(datetime.strptime(times[0]['begin'], '%Y%m%dT%H%M%S').isoformat() + '+02:00')
    end.append(datetime.strptime(times[0]['end'], '%Y%m%dT%H%M%S').isoformat() + '+02:00')
    
    ### Cause and Effect
    cause.append(row['cause'])
    effect.append(row['effect'])
    
merged_df['begin'] = begin
merged_df['end'] = end
merged_df['cause'] = cause
merged_df['effect'] = effect

merged_df = merged_df.drop(columns=['outage_data'])

In [ ]:
merged_df.head()

In [ ]:
# plot outages on a heatmap
m = folium.Map(location=[45.75, 4.85], zoom_start=13, tiles='cartodb voyager')

# heatmap
heat_data = [[row['lat'], row['lon']] for _, row in merged_df.iterrows()]
HeatMap(heat_data).add_to(m)

# marker cluster
marker_cluster = MarkerCluster().add_to(m)
for idx, row in merged_df.iterrows():
    tooltip = f"Stop Name: {row['stop_name']}<br>Outage ID: {row['outage_id']}<br>Start: {row['begin']}<br>End: {row['end']}<br>Cause: {row['cause']}<br>Effect: {row['effect']}"
                
    folium.Marker([row['lat'], row['lon']], tooltip=tooltip).add_to(marker_cluster)

# tcl metro lines
folium.GeoJson(tcl_metro, style_function=lambda x: x['properties']['style']).add_to(m)


'''
Outage database only contains data on metro line/stop outages, so bus lines are not displayed by default.
The bus line data is still useful to see how metro lines are connected to bus lines and thus how outages on metro lines could affect bus lines.
Uncomment the following lines to display the bus lines.
'''
# tcl_bus = gpd.read_file('data/tcl_bus.json')
# tcl_bus = tcl_bus.drop(columns=['date_debut', 'date_fin', 'last_update', 'last_update_fme'])
# folium.GeoJson(tcl_bus, style_function=lambda x: {'color': '#555', 'weight': 1.5, 'dashArray': '5, 5'}).add_to(m)

m.save('outages_heatmap.html')